In [2]:
'''
uses objects to calculate iv2 and then puts those into dataframe and list
'''
import os
import pickle
import pandas as pd
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt
import obspy
import earthquake
import util


from matplotlib.widgets import PolygonSelector
from matplotlib.path import Path
from obspy.clients.fdsn import Client
import cartopy.crs as ccrs
import matplotlib.patches as patches
import pandas as pd
import cartopy.feature as cf

from datetime import date

#+++++++PLOTTING++++++++++
# LINE_BY_MAG_TYPE = True
# COLOR_BY_MAG_TYPE = True

In [3]:
list_mag = []
list_dist = []
list_iv2 = []
list_mag_types = []

ROOT = '/home/earthquakes1/homes/Rebecca/phd/data/2019_global_m3/'
eq_list = os.listdir(ROOT)
cat = obspy.read_events('/home/earthquakes1/homes/Rebecca/phd/data/2019_global_m3_catalog.xml')

eq_with_data = []
cat_with_data = obspy.Catalog()  # cat.copy()
# cat_with_data.clear()
for event in cat:  # check earthquakes have data AND PICKS
    eq_name = util.catEventToFileName(event)
    if (os.path.isdir(ROOT+eq_name) and
            os.path.isdir(ROOT+eq_name+'/station_xml_files') and
            os.path.exists(ROOT+eq_name+'/picks.pkl')):
        eq_with_data.append(eq_name)
        cat_with_data.extend([event])

In [4]:
cat_with_data[0].magnitudes[0].magnitude_type

'mb'

In [5]:
to_plot_list = []
for cat_entry in cat_with_data:
        to_plot_list.append([cat_entry.origins[0].latitude, cat_entry.origins[0].longitude, cat_entry.origins[0].depth, cat_entry.magnitudes[0].mag, cat_entry.magnitudes[0].magnitude_type[0:2].lower(), cat_entry.origins[0].time])
to_plot=pd.DataFrame(to_plot_list, columns = ['lat', 'long', 'depth', 'mag', 'mag_type', 'time'])

In [6]:
to_plot

,lat,long,depth,mag,mag_type,time
0,-20.486500,-69.016500,102120.0,4.20,mb,2019-01-31T19:49:49.660000Z
1,8.978200,-83.757800,13590.0,4.90,mw,2019-01-31T13:09:53.370000Z
2,56.298000,-149.259000,19350.0,4.10,ml,2019-01-31T13:03:50.890000Z
3,17.957100,-68.394500,52000.0,3.09,md,2019-01-31T10:16:56.840000Z
4,53.522800,-167.269200,8590.0,3.00,ml,2019-01-31T08:55:20.080000Z
...,...,...,...,...,...,...
3381,35.787000,-117.619333,3010.0,3.68,ml,2019-07-06T06:47:08.090000Z
3382,35.784667,-117.601833,5000.0,3.44,ml,2019-07-06T06:47:07.980000Z
3383,-18.441600,176.438400,10000.0,4.60,mb,2019-07-06T06:41:16.710000Z
3384,35.800835,-117.633499,6550.0,3.90,ml,2019-07-06T06:41:15.080000Z


In [7]:
cat_entry.__dict__

{'resource_id': smi:service.iris.edu/fdsnws/event/1/query?eventid=11058889,
 'event_type': 'earthquake',
 'event_type_certainty': None,
 'creation_info': None,
 'preferred_origin_id': smi:service.iris.edu/fdsnws/event/1/query?originid=39464688,
 'preferred_magnitude_id': smi:service.iris.edu/fdsnws/event/1/query?magnitudeid=195053049,
 'preferred_focal_mechanism_id': None,
 'event_descriptions': [EventDescription(text='CENTRAL CALIFORNIA', type='Flinn-Engdahl region')],
 'comments': [],
 'picks': [],
 'amplitudes': [],
 'focal_mechanisms': [],
 'origins': [Origin
	   resource_id: ResourceIdentifier(id="smi:service.iris.edu/fdsnws/event/1/query?originid=39464688")
	          time: UTCDateTime(2019, 7, 6, 6, 34, 47, 460000)
	     longitude: -117.726669
	      latitude: 35.895168
	         depth: 7470.0
	 creation_info: CreationInfo(author='ci,us')],
 'magnitudes': [Magnitude
	    resource_id: ResourceIdentifier(id="smi:service.iris.edu/fdsnws/event/1/query?magnitudeid=195053049")
	      

In [8]:
fig

NameError: name 'fig' is not defined

In [ ]:
        colors = ['#ed8b00','#dd4124','#0f85a0','#edd746']
        #colors = ['#625a95','#cc5c75','#f47947','#f7ac2b']
        groups = to_plot.groupby(['mag_type'])
        for  j,(k,v) in enumerate(groups):
            axs.scatter(np.array(v.mag)+np.transpose(np.random.uniform(-0.05, 0.05, size=(len(v.mag),1))), np.log10(v.iv2*(v.dist**2)),
                       color = colors[j],
                       alpha = 0.5,
                       marker = '.',
                       label = k + ',' + str(len(v.mag_type)))
            dist_corr_mult = v.iv2*(v.dist**2)
            mag_plot = v.mag
            initial_guess = (1.4,0)
            res = optimize.minimize(obj, initial_guess, method = 'Nelder-mead')
            def obj(to_opt):
                '''optimisation function'''
                a_opt=to_opt[0]
                b_opt=to_opt[1]
                #importance = np.array(n)
                y_real= np.log10(dist_corr_mult)
                x_list = np.array(mag_plot)
                y_guess = (a_opt*x_list+b_opt)
                #print(y_guess)
                return sum(abs(y_guess-y_real))
            x = np.arange(min(mag_plot), max(mag_plot), 0.1)
            y = res.x[0] * x + res.x[1]
            axs.plot(x,y, label = str(res.x[0]) + '*x+'+ str(res.x[1]), color = colors[j])

In [20]:
%matplotlib qt
import cartopy.feature as cfeature
colors = {'ml':'#0f85a0','mw':'#edd746','mb':'#ed8b00','md':'#dd4124'}
import matplotlib as mpl#markers = {'ml':'x','mw':'o','mb':'x','md':'^'}
valid_markers = ([item[0] for item in mpl.markers.MarkerStyle.markers.items() if item[1] is not 'nothing' and not item[1].startswith('tick') and not item[1].startswith('caret')])
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(1,1,1,projection=ccrs.PlateCarree())
ax.coastlines()
ax.add_feature(cf.BORDERS)
ax.add_feature(cfeature.OCEAN, color = 'silver', alpha = 0.4)
ax.add_feature(cfeature.LAND,color='white')
colors = {'ml':'#0f85a0','mw':'#edd746','mb':'#ed8b00','md':'#dd4124'}
#pts = ax.scatter(to_plot['long'], to_plot['lat'], c = list(to_plot.mag_type.map(colors)), marker = 'o', alpha = 0.5)
colors = ['#ed8b00','#dd4124','#0f85a0','#edd746']
groups = to_plot.groupby(['mag_type'])
markers = ['o','x','^','s']
zorders = [0,3,1,2]
for  j,(k,v) in enumerate(groups):
    ax.scatter(v.long,v.lat,
               color = colors[j],
               alpha = 1,
               marker = markers[j],
               s = 20,
               zorder = zorders[j],
               label = k + ',' + str(len(v.mag_type)),
              edgecolors='none')
im_ratio = ax.bbox.width/ax.bbox.height
#plt.colorbar(pts, fraction=0.01*im_ratio, pad=0.04, label='mag')
#for key in colors.keys():
#    ax.scatter(180,0, c = colors[key], label = key, marker = 'o')
plt.legend()

<>:5: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:5: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/tmp/ipykernel_697096/1412807121.py:5: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  valid_markers = ([item[0] for item in mpl.markers.MarkerStyle.markers.items() if item[1] is not 'nothing' and not item[1].startswith('tick') and not item[1].startswith('caret')])
/home/earthquakes1/software/Mageia8/anaconda3/lib/python3.9/site-packages/cartopy/mpl/geoaxes.py:1546: UserWarning: You passed a edgecolor/edgecolors ('none') for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  result = matplotlib.axes.Axes.scatter(self, *args, **kwargs)


In [108]:
j

3

In [22]:
# reference: https://mentalitch.com/key-events-in-rock-and-roll-history/
from datetime import timedelta
dates = to_plot['time']
min_date = date(np.min(dates).year, np.min(dates).month, np.min(dates).day)-timedelta(days = 5)
max_date = date(np.max(dates).year, np.max(dates).month, np.max(dates).day)+timedelta(days = 20)

In [23]:
dates

0       2019-01-31T19:49:49.660000Z
1       2019-01-31T13:09:53.370000Z
2       2019-01-31T13:03:50.890000Z
3       2019-01-31T10:16:56.840000Z
4       2019-01-31T08:55:20.080000Z
                   ...             
3381    2019-07-06T06:47:08.090000Z
3382    2019-07-06T06:47:07.980000Z
3383    2019-07-06T06:41:16.710000Z
3384    2019-07-06T06:41:15.080000Z
3385    2019-07-06T06:34:47.460000Z
Name: time, Length: 3386, dtype: object

In [24]:
np.random.rand()

0.4633622782315042

In [25]:
fig, ax = plt.subplots(figsize=(15, 4), constrained_layout=True)
#_ = ax.set_ylim(-0.1, 1.1)
_ = ax.set_xlim(min_date, max_date)
#_ = ax.axhline(0, xmin=0.05, xmax=0.95, c='deeppink', zorder=1)
y = np.random.rand(len(dates))
#_ = ax.scatter(dates, y, s=120, c = list(to_plot.mag_type.map(colors)), marker = '.')

colors = ['#ed8b00','#dd4124','#0f85a0','#edd746']
groups = to_plot.groupby(['mag_type'])
markers = ['o','x','^','s']
zorders = [0,3,1,2]

for  j,(k,v) in enumerate(groups):
    ax.scatter(v.time,np.zeros(len(v.time))+j+np.random.rand(len(v.time)),
               color = colors[j],
               alpha = 1,
               marker = markers[j],
               s = 30,
               zorder = zorders[j],
               label = k + ',' + str(len(v.mag_type)))
plt.legend()
plt.yticks([])

([], [])

In [83]:
np.random.rand(len(dates))

array([ 0.04524652,  0.7535646 ,  0.7276717 , ...,  0.3035848 ,
        0.37905991,  0.94968318])